# Trabalho Prático 1 - Aprendizado de Máquina
<font size=4>Nome: Alexandre Maros<br></font>
<font size=3>UFMG - 2018/1</font>


Blabla...

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import os

pd.options.display.max_rows = 10

# 1. Leitura dos Dados

In [ ]:
# Encontrar Diretorio Atual
diretorio_atual = os.path.realpath('.')

# Ler o arquivo de entrada
dados = pd.read_csv(os.path.join(diretorio_atual, "data_tp1"), sep=",", header=None)

# Armazenar o label correto dos numeros
dados = dados.rename(columns = {0:'label'})
y = dados.label

# Retirar a primeira coluna referente aos labels
X = dados.drop("label", axis=1)

# Modificar o nome das colunas
num_imagens = X.shape[1]
X.columns = [ int(x) for x in range(0, num_imagens)] 

X

# 2. Visualização dos dados

Nesta etapa...

### Visualização dos três primeiro dígitos do dataset

In [ ]:
digitos = [ X.loc[0,:], X.loc[1,:], X.loc[2,:] ]
labels = [ y[0], y[1], y[2] ]

# 784 colunas correspondem a uma imagem de 28x28
plot1 = np.reshape(digitos[0].values, (28, 28))
plot2 = np.reshape(digitos[1].values, (28, 28))
plot3 = np.reshape(digitos[2].values, (28, 28))

fig=plt.figure(figsize=(8, 8))

# Plot the digits
fig.add_subplot(4, 4, 1)
plt.imshow(plot1, cmap='gray_r')
plt.title('Label: {}'.format(labels[0]))

fig.add_subplot(4, 4, 2)
plt.imshow(plot2, cmap='gray_r')
plt.title('Label: {}'.format(labels[1]))

fig.add_subplot(4, 4, 3)
plt.imshow(plot3, cmap='gray_r')
plt.title('Label: {}'.format(labels[2]))

plt.show()


### Verificando a distribuição dos dígitos

É interessante verificar a disposição (frequência) dos dígitos no dataset. Se tiver muitos digitos 1 e poucos dígitos 9 isso pode levar a uma má previsão.

In [ ]:
y.value_counts(normalize=True).sort_index()

Os dígitos parecem estar bem distribuídos, sendo que os dígitos 0 e 5 são os dois digitos que menos aparecem (9.2% dos dígitos são o dígito 0 e 0.912% dos digitos são o dígito 5). O dígito 1 é o que mais aparece, com sua frequência em 11%.

# 3. Construção dos Modelos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
scaler.fit(X)
StandardScaler(copy=True, with_mean=True, with_std=True)
X = scaler.transform(X)

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y,
        cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='neg_log_loss')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


In [ ]:
clf = MLPClassifier(solver='sgd', activation='logistic', batch_size='auto', early_stopping=False,
                    hidden_layer_sizes=(50,), learning_rate_init=0.01, max_iter=200, momentum=0.5,
                    shuffle=True, verbose=False)

plot_learning_curve(clf, "Titulo", X, y)

In [ ]:
def calculate_scores_varying_lr(title, X, y, hidden_layer_sizes, batch_size, ylim=None, cv=10,
                        scoring='neg_log_loss', n_jobs=1, train_sizes=np.linspace(.1, 1.0, 10)):

    types = ['o-', '+-', 'D-']
    t = 0
    
    scores = []
    for i in [0.01, 0.1, 0.5]:
        
        estimator = MLPClassifier(solver='sgd', activation='logistic', batch_size=batch_size, early_stopping=False,
                    hidden_layer_sizes=hidden_layer_sizes, learning_rate_init=i, max_iter=200, momentum=0.9,
                    shuffle=True, verbose=False)
        
        train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, 
            cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring=scoring)
        
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        
        scores.append({'learning_rate': i,
                       'batch_size': batch_size,
                       'hidden_layer_sizes': hidden_layer_sizes,
                       'legend_type': types[t],
                       'train_sizes': train_sizes,
                       'train_scores': train_scores,
                       'test_scores': test_scores,
                       'train_scores_mean': train_scores_mean,
                       'train_scores_std': train_scores_std,
                       'test_scores_mean': test_scores_mean,
                       'test_scores_std': test_scores_std})

        t = t + 1

    return scores

In [ ]:
# Variaveis
# batch_size, hidden_layer_sizes, learning_rate_init, max_iter (num epochs)

#print("1.")
#mlp_25_minibatch50 = calculate_scores_varying_lr("Teste", X, y, 25, 50)
#print("2.")
#mlp_50_minibatch50 = calculate_scores_varying_lr("Teste", X, y, 50, 50)
#print("3.")
#mlp_100_minibatch50 = calculate_scores_varying_lr("Teste", X, y, 100, 50)

In [ ]:
def plot_scores(ax, plot):
    l = []
    for i in (0, 1, 2):
        ax.fill_between(plot[i]['train_sizes'], plot[i]['train_scores_mean'] - plot[i]['train_scores_std'],
                            plot[i]['train_scores_mean'] + plot[i]['train_scores_std'], alpha=0.1,
                             color="r")
        ax.fill_between(plot[i]['train_sizes'], plot[i]['test_scores_mean'] - plot[i]['test_scores_std'],
                         plot[i]['test_scores_mean'] + plot[i]['test_scores_std'], alpha=0.1, color="g")
        leg1, = ax.plot(plot[i]['train_sizes'], plot[i]['train_scores_mean'], plot[i]['legend_type'], color="r",
                 label="Training score " + str(i))
        leg2, = ax.plot(plot[i]['train_sizes'], plot[i]['test_scores_mean'], plot[i]['legend_type'], color="g",
                     label="Cross-validation score" + str(i))
        
        l.append(leg1)
        l.append(leg2)
        
        ax.set_xlabel('Batch Size: ' + str(plot[i]['batch_size']), fontsize=18)
        ax.set_ylabel('Neuronios: ' + str(plot[i]['hidden_layer_sizes']), fontsize=18)
        
        ax.ylim=(-3, 0)
        
    return l

In [ ]:
plt.close('all')

fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

from mpl_toolkits.axes_grid1 import Grid
grid = Grid(fig, rect=111, nrows_ncols=(3,3),
            axes_pad=0.25, label_mode='L',
            )

i = 0
#plots = [mlp_25_minibatch50, mlp_50_minibatch50, mlp_100_minibatch50, 
#         mlp_25_minibatch50, mlp_50_minibatch50, mlp_100_minibatch50, 
#         mlp_25_minibatch50, mlp_50_minibatch50, mlp_100_minibatch50]
l = []
for ax in grid:
    l = plot_scores(ax, mlp_25_minibatch50[i])
    ax.title.set_visible(False)
    i = i + 1

fig.legend(l, l, 'upper right')
    
plt.tight_layout()

In [ ]:
#clf.predict_proba(X.loc[1,:].values.reshape(1, -1))

#clf.fit(X_train, y_train)
#predictions = clf.predict_proba(X_test)
#log_loss(y_test, predictions)